# Fine-tuned RoBERTa for multi-label text classification in Spanish (CPV codes)

This notebook follows the steps in the tutorial by Niels Rogge https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb, originally written for BERT and adapted for the CPV multilabel classification problem in the Spanish language (using RoBERTa trained in the BNE corpus) by María Navas-Loro.


## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [ ]:
!pip install -q transformers datasets
!pip install -y ipywidgets
!pip uninstall -y jupyterlab_widgets
!pip install -y jupyterlab_widgets
!pip install huggingface_hub

In [ ]:
!sudo apt-get install git-lfs

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



# Data preparation

We prepare our datasets, splitting the CPV column in 45 different columns with binary values.

In [ ]:
import pandas as pd

In [ ]:
# We read the input data
df = pd.read_csv('../input/dataset/train.csv')
dftest = pd.read_csv('../input/dataset/test.csv')

#df = pd.read_csv('../input/dataset10/train10.csv')
#dftest = pd.read_csv('../input/dataset10/test10.csv')

#dftest['descripcion'] = dftest['descripcion'].apply(lambda x: x.strip('"'))
#df['descripcion'] = df['descripcion'].apply(lambda x: x.strip('"'))

df.pop('Unnamed: 0')
dftest.pop('Unnamed: 0')
df.head()


In [ ]:
labels = df.columns[1:]

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

## Preprocess data

Now we have the data ready, we have to tokenize it so the model can read it.

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")

import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

datasettrain = Dataset(pa.Table.from_pandas(df))

datasettest = Dataset(pa.Table.from_pandas(dftest))

def preprocess_data(df):
  # take a batch of texts
  text = df["descripcion"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: df[k] for k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = datasettrain.map(preprocess_data, batched=True, remove_columns=datasettrain.column_names)

In [ ]:
encoded_dataset_test = datasettest.map(preprocess_data, batched=True, remove_columns=datasettest.column_names)

In [ ]:
# To check an example
#example = encoded_dataset[0]
#print(example.keys())

In [ ]:
#tokenizer.decode(example['input_ids'])

In [ ]:
# Labels of this example
#example['labels']

In [ ]:
# To translate this output to the actual positive labels
# [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

In [ ]:
encoded_dataset.set_format("torch")

## Define model

We will now finetune the RoBERTa model for Spanish trained on the BNE corpus, adding a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("PlanTL-GOB-ES/roberta-base-bne", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"roberta-finetuned-CPV_Spanish",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_average_precision_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    coverage_err = coverage_error(y_true, y_pred)
    label_ranking_average_precision = label_ranking_average_precision_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'coverage_error': coverage_err,
               'label_ranking_average_precision_score': label_ranking_average_precision}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Here is where the real training begins.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import os
import wandb

wandb.init()
#os.environ["WANDB_DISABLED"] = "true"

trainer.train()

## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

## Inference

Let's test the model on a new sentence:

In [ ]:
text = "Ejecución de obras de almacén en el CEIP San Miguel Arcángel"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

In [ ]:
#trainer.save_model("./trainer")

<a href="./trainer/pytorch_model.bin"> Download model </a><br>
<a href="./trainer/config.json"> Download config </a><br>
<a href="./trainer/tokenizer_config.json"> Download tokenizer_config </a><br>
<a href="./trainer/special_tokens_map.json"> Download special_tokens_map </a>


<a href="./trainer10/pytorch_model.bin"> Download model 10 </a><br>
<a href="./trainer10/config.json"> Download config 10</a><br>
<a href="./trainer10/tokenizer_config.json"> Download tokenizer_config 10</a><br>
<a href="./trainer10/special_tokens_map.json"> Download special_tokens_map 10</a>

In [ ]:
trainer.push_to_hub()